In [3]:
import pickle
embedding_cache_path = "/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/dictionary_with_embeddings.csv"
embedding_cache = pickle.load(open(embedding_cache_path, "rb"))

EOFError: Ran out of input

## Code

In [45]:
from tqdm import tqdm
from dotenv import load_dotenv
import openai
import pickle
import pandas as pd

In [46]:
load_dotenv()
openai_client = openai.OpenAI()

### Utils

In [47]:
MAX_BATCH_SIZE = 2048

def get_embeddings(list_of_text: list[str], model="text-embedding-ada-002", **kwargs) -> list[list[float]]:
    assert len(list_of_text) <= 2048, "The batch size should not be larger than 2048."

    # replace newlines, which can negatively affect performance.
    list_of_text = [text.replace("\n", " ") for text in list_of_text]

    data = openai.embeddings.create(input=list_of_text, model=model, **kwargs).data
    return [d.embedding for d in data]

def save_embeddings(embedding_cache, embedding_cache_path):
    with open(embedding_cache_path, "wb") as embedding_cache_file:
            pickle.dump(embedding_cache, embedding_cache_file)

def update_embedding_cache(words, embeddings, embedding_cache):
    for word, embedding in zip(words, embeddings):
        embedding_cache[word] = embedding
    return embedding_cache

def run_batch(words, embedding_cache, embedding_cache_path):
    batch_words = get_words_without_embeddings(words, embedding_cache)[:MAX_BATCH_SIZE]
    batch_embeddings = get_embeddings(batch_words)
    embedding_cache = update_embedding_cache(batch_words, batch_embeddings, embedding_cache)
    save_embeddings(embedding_cache, embedding_cache_path)
    words_remaining = len(get_words_without_embeddings(words, embedding_cache))
    if words_remaining > 0: print(f"Words to embed remaining: {words_remaining}"); return True 
    return False

def get_words_without_embeddings(words, embedding_cache):
    lst_embed_cache = list(embedding_cache.keys())
    return [v for v in words if v not in lst_embed_cache]

In [48]:
# Get all embeddings:
embedding_cache_path = "/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/dictionary_with_embeddings.pickle"
words = sorted(pickle.load(open("/Users/lucasvilsen/Desktop/GrammatikTAK/Datasets/dictionary.pickle", "rb")))
embedding_cache = pickle.load(open(embedding_cache_path, "rb"))
batches_to_run = (len(words) - len(embedding_cache)) // 2048

In [49]:
to_continue = True
while to_continue:
    to_continue = run_batch(words, embedding_cache, embedding_cache_path)

KeyboardInterrupt: 

In [28]:
len(embedding_cache)

2010